In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!wget 'https://firebasestorage.googleapis.com/v0/b/hseye-fff80.appspot.com/o/144maps.npy?alt=media&token=c0645160-cd9e-44fa-b6d0-54bb56159cf5'

In [ ]:
!wget 'https://firebasestorage.googleapis.com/v0/b/hseye-fff80.appspot.com/o/20kGauss.npy?alt=media&token=8658a6f2-57b2-4877-8041-17ad52c9e139'

%load_ext tensorboard


In [ ]:
#'https://firebasestorage.googleapis.com/v0/b/homesteadfbla-31a97.appspot.com/o/maps.mat?alt=media&token=37b8090b-1030-4167-8f8b-6c3bbf6a2760', 'https://firebasestorage.googleapis.com/v0/b/hseye-fff80.appspot.com/o/RealData.zip?alt=media&token=7656e1e7-836b-4fdf-adc0-cff354a93b9c', 'https://firebasestorage.googleapis.com/v0/b/secrets-91dd7.appspot.com/o/144maps.mat?alt=media&token=f3efe84d-07cd-49f9-ab40-983d7011c70f'

In [ ]:
!pip install tensorflow

In [2]:
!wget 'https://firebasestorage.googleapis.com/v0/b/hseye-fff80.appspot.com/o/20kGauss.npy?alt=media&token=8658a6f2-57b2-4877-8041-17ad52c9e139'

--2022-07-10 00:53:27--  https://firebasestorage.googleapis.com/v0/b/hseye-fff80.appspot.com/o/20kGauss.npy?alt=media&token=8658a6f2-57b2-4877-8041-17ad52c9e139
Resolving firebasestorage.googleapis.com (firebasestorage.googleapis.com)... 142.251.8.95, 2404:6800:4008:c15::5f
Connecting to firebasestorage.googleapis.com (firebasestorage.googleapis.com)|142.251.8.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3317760128 (3.1G) [application/octet-stream]
Saving to: ‘20kGauss.npy?alt=media&token=8658a6f2-57b2-4877-8041-17ad52c9e139’

20kGauss.npy?alt=me 100%[===================>]   3.09G  91.2MB/s    in 35s     

2022-07-10 00:54:04 (90.2 MB/s) - ‘20kGauss.npy?alt=media&token=8658a6f2-57b2-4877-8041-17ad52c9e139’ saved [3317760128/3317760128]



In [ ]:
np.load("20kGauss.npy")

In [ ]:
import shutil
shutil.unpack_archive("RealData.zip")

In [ ]:
!pip install firebase_admin

In [2]:
from Model import Model
from Losses import Losses
from ModelEvaluation import ModelEvaluation
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import scipy.io
import numpy as np
from DataGenerator import DataGenerator
from TrainingManager import TrainingManager
from Database import Database
# from VidTrainingManager import VidTrainingManager
from VideoGenerator import VideoGenerator


In [3]:
generator = VideoGenerator(topNm = 2000)


In [4]:
images, cleanImages, maps = generator.generateVideos(5)

  0%|          | 0/5 [00:00<?, ?it/s]

status 0


 20%|██        | 1/5 [00:02<00:08,  2.01s/it]

status 1


 40%|████      | 2/5 [00:02<00:03,  1.16s/it]

...

 60%|██████    | 3/5 [00:03<00:01,  1.13it/s]

status 3


 80%|████████  | 4/5 [00:03<00:00,  1.32it/s]

status 4


100%|██████████| 5/5 [00:04<00:00,  1.17it/s]


In [5]:
def auxSlice(content, channels):
    numVid = content.shape[0]
    numFrame = content.shape[1]
    width = content.shape[2]
    height = content.shape[3]
    
    
    slices = np.zeros((numVid, numFrame*width, height, channels))
        
    for i in range(numVid):
        stack = []
        for k in range(numFrame):
            stack.append(content[i][k].reshape(width, height, channels))
        slices[i] = np.vstack(stack)
        
        
    return slices

        

In [6]:
from matplotlib import pyplot as plt
import heapq
import csv
from matplotlib import cm

def showImage(image):
    data = np.reshape(image, (image.shape[0],image.shape[1],3))
    plt.imshow(imRegulate(data), interpolation='nearest')
    return plt.show()
    
    
def showMap(depthMap):

    d1 = depthMap.shape[0]
    d2 = depthMap.shape[1]
    
    pData = np.reshape(depthMap, (d1,d2))

    fig = plt.figure(figsize = (5,5))
    ax1 = fig.add_subplot(projection='3d')

    x = np.arange(0, d1)
    y = np.arange(0, d2)
    
    X, Y = np.meshgrid(x, y)
    surf = ax1.plot_surface(X, Y, pData, cmap = cm.coolwarm)

    ax1.set_zlabel("Thickness" + " (" + "nm" + ")")
    ax1.set_title('Patient Tear Film Thickness Profile')



    return plt.show()
    
        
def imRegulate(data):
    m = np.max(data)
    mi = np.min(data)
    norm = ((data - mi) / (m - mi))*255
    return norm.astype(np.uint8)


def tRegulate(data):
    data = tf.truediv(
       tf.subtract(
          data, 
          tf.reduce_min(data)
       ), 
       tf.subtract(
          tf.reduce_max(data), 
          tf.reduce_min(data)
       )
    )
    return tf.math.multiply(data,5000)

@tf.custom_gradient
def tCycle(depth):
    depth = tRegulate(depth)
    depth = tf.convert_to_tensor(depth)
    depth = tf.cast(tf.reshape(depth, (48*48)), tf.int32).numpy()
    print(depth)

    def grad(upstream):
            values = dColorMap[[depth]]
            ret = tf.convert_to_tensor(values.reshape(1,48,48,3).astype(np.float32))
            return ret * upstream
    
    values = colorMap[[depth]]
    return tf.convert_to_tensor(values.reshape(1,48,48,3).astype(np.float32)), grad


In [ ]:
maps = np.array(scipy.io.loadmat('LARGEmaps.mat')['maps'])

In [ ]:
import os
path = "/opt/conda/lib/python3.7/site-packages/models/images/"
counter = 1
for f in os.listdir(path):
    suffix = f.split('.')[-1]
    if suffix == 'jpg' or suffix == 'png':
        new = '{}.{}'.format(str(counter), suffix)
        os.rename(path + f, path + new)
        counter = int(counter) + 1

In [ ]:
database = Database()

In [ ]:
losses = Losses()

In [ ]:
noisyRGB = {
    "gen_down" : [512,1024,2048],
    "gen_up": [512,1024,2048],
    "gen_dropout" : [True, False, False],
    "disc_down": [32, 64, 128, 256],
    "gen_lr": 0.002,
    "gen_b1": 0.5,
    "disc_lr" : 0.002,
    "disc_b1": 0.5,
    "epochs": 50,
    "time": database.fetchDate(),
    "name" : "longtest10/",
    "hsi" : False,
    "numPerlin": 5000,
    "numGaussian": 20000,
    "topNm": 2000,
    "notes": "Noisy, 0.1",
    "optimalCheckpoint": 10,
    "std": 0.1,
    "d1": 1728,
    "d2": 144
}

In [ ]:
vidRGB = {
    "gen_down" : [128,256,512],
    "gen_up": [128,256,512],
    "gen_dropout" : [True, True, False],
    "disc_down": [32],
    "gen_lr": 0.0002,
    "gen_b1": 0.5,
    "disc_lr" : 0.0002,
    "disc_b1": 0.5,
    "epochs": 6,
    "time": database.fetchDate(),
    "name" : "longtest1",
    "topNm": 5000,
    "notes": "Noisy, 0.1",
    "optimalCheckpoint": 10,
    "numVid": 10, 
    "d1": 1728, 
    "d2": 144
}

In [ ]:
noisyRGBManager = TrainingManager(noisyRGB, auxSlice(images, 3), auxSlice(maps, 1), auxSlice(images, 3) ,auxSlice(maps, 1))

In [ ]:
noisyRGBManager.previewGenerator()

In [ ]:
noisyRGBManager.previewDiscriminator()

In [ ]:
noisyRGBManager.fit()

In [ ]:
#noisyRGBManager.fit()

In [ ]:
# q1m = TrainingManager(q1)
# q2m = TrainingManager(q2)
q4m = TrainingManager(q4)


In [ ]:
from CrossModelEvaluator import CrossModelEvaluator

In [ ]:
cross = CrossModelEvaluator()

In [ ]:
cross.compareModels(["noisy_hsi_1", "noisy_rgb_1"], file = "test7.csv", stds = [0.4,0.5,0.6], checkpoint = 6)

In [ ]:
    def fetchManagers(sessions, checkpoint):
        managers = []
        for session in sessions:
            manager = TrainingManager(database.fetchModel(session))
            if checkpoint == -1:
                manager.restoreLatestCheckpoint()
            else:
                manager.restoreCheckpoint(checkpoint)
            
            managers.append(manager)
        return managers

In [ ]:
managers = fetchManagers(["second_train_mixed_hsi"], -1)
